# History

## Test with and without scaling
- Clean and (vanilla) StandardScaler() min_interval=20 => 0.63142 (less than without scaling/Cleaning) overfit test set?
  - 0.9670995670995671 On the reduce_test_set 
  
- Clean min_interval=20 => 0.63142... Same?  (normal, in random forest it is *useless*)
  - 0.9783549783549783 On the reduce_test_set 

## Params 
- Grid_search "classify__n_estimator": [3, 10, 30], "classify__min_interval": [3, 5, 10]}

## Same n° of classes (yay)

## Not enough tunable parameters in sktime lets do our own model
- Give quite good score (window=16) (less than sktime but still good): 0.9714285714285714 (Better score on Kaggle !!!) - give much more better result by removing nAN (0.73 on kaggle)
- Perform better than a *naive* randomForest given X_train_reduce/Y_train_reduce: 0.9428571428571428
- On the reduce test set having a window=4 give *worst result:* 0.9506493506493506
- Time series model perform better 

## Leave one out group => more accurate according to the test set than using typical grid search
## Worst result but more accurate
## No nan o test set, can do better preprocess by doing the median only on the same calss
## Trying with takeing the median for a given class but perform quite bad

In [40]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.transformations.panel.compose import ColumnConcatenator
from python.utils import load_data
from python.preprocess import TimeSerieMaker, CleanNanByMedian, SimpleSlidingWindowFeaturesExtractor
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sktime.datatypes._panel._convert import (
    from_3d_numpy_to_nested,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from python.model_selection import crossValidationOneOut, GridSearchGroup
from sklearn.impute import SimpleImputer

# Data

In [71]:
DATA_PATH = 'data'
X_train, y_train, X_test, subjects = load_data(DATA_PATH)
X_train = SimpleImputer(missing_values=-999999.99, strategy="median").fit_transform(X_train) # take the median instead of Nan

X_train size: (3500, 15872).
y_train size: (3500,).
X_test size: (3500, 15872).


In [4]:
X_train_reduce, X_test_reduce, y_train_reduce, y_test_reduce = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Models

In [ ]:
best_model = Pipeline([
    ("std_scaler", StandardScaler()),
    ("sswfe", SimpleSlidingWindowFeaturesExtractor(window_size=16)),
    ("rnf_cls", RandomForestClassifier(random_state=42, n_estimators=200))
])

In [84]:
from sklearn.neighbors import KNeighborsClassifier

pipeline_own_knn = Pipeline([
    ("std_scaler", StandardScaler()),
    ("sswfe", SimpleSlidingWindowFeaturesExtractor(window_size=16)),
    ("rnf_cls", KNeighborsClassifier( n_jobs=-1))
])

In [87]:
from sklearn.linear_model import LogisticRegression

pipeline_own_log = Pipeline([
    ("std_scaler", StandardScaler()),
    ("sswfe", SimpleSlidingWindowFeaturesExtractor(window_size=16)),
    ("rnf_cls", LogisticRegression(random_state=42))
])

In [72]:
pipeline_own_rf = Pipeline([
    ("std_scaler", StandardScaler()),
    ("sswfe", SimpleSlidingWindowFeaturesExtractor(window_size=16)),
    ("rnf_cls", RandomForestClassifier(random_state=42))
])

In [90]:
from sklearn.ensemble import VotingClassifier

voting_clf = Pipeline([
    ("std_scaler", StandardScaler()),
    ("sswfe", SimpleSlidingWindowFeaturesExtractor(window_size=16)),
    ("vot", VotingClassifier(
        estimators=[("lr",LogisticRegression()),
                    ("rnf", RandomForestClassifier(random_state=42, n_estimators=200, max_features=2, bootstrap=False, max_depth=80)),
                    ("knn", KNeighborsClassifier(n_jobs=-1))], 
        voting="soft"
    )) 
])

In [17]:
column_concatenator = [
    ("std_scaler", StandardScaler()),
    ("time-serie", TimeSerieMaker()),
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, min_interval=3)),
]

time_serie_forest_clf = Pipeline(column_concatenator)

# Search parameters

In [73]:
param_grid = [
    {"rnf_cls__n_estimators": [150], "rnf_cls__max_features": [2, 4, "auto"], 'rnf_cls__bootstrap': [True, False], 'rnf_cls__max_depth': [5, 8, 15, 25], "sswfe__window_size": [16, 32]}
]

gsg = GridSearchGroup(pipeline_own_rf, param_grid)

In [74]:
gsg.fit(X_train, y_train, subjects)

In [ ]:
cvres = gsg._cv_results
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

{'mean_test_score': 0.5116985352555026, 'params': (200, 2, True, 110)}

# Scores

In [12]:
pipeline_own_rf.fit(X_train, y_train) # best model

Pipeline(steps=[('std_scaler', StandardScaler()),
                ('sswfe', SimpleSlidingWindowFeaturesExtractor(window_size=16)),
                ('rnf_cls',
                 RandomForestClassifier(max_features=4, n_estimators=200,
                                        random_state=42))])

In [92]:
voting_clf.score(X_test_reduce, y_test_reduce)

0.9645021645021645

[0.78,
 0.5783475783475783,
 0.6802197802197802,
 0.4087237479806139,
 0.6692056583242655]

[0.8085714285714286,
 0.6951566951566952,
 0.4989010989010989,
 0.518578352180937,
 0.5614798694232862]

In [9]:
crossValidationOneOut(X_train, y_train, pipeline_own_rf, subjects)

[0.8085714285714286,
 0.6951566951566952,
 0.4989010989010989,
 0.518578352180937,
 0.5614798694232862]

In [ ]:
time_serie_forest_clf.fit(X_train_reduce, y_train_reduce) # sktime (perform worst on kaggle)

In [18]:
time_serie_forest_clf.score(X_test_reduce, y_test_reduce)

0.9774891774891775

# Submit prediction

In [13]:
final_model = pipeline_own_rf
y_pred = final_model.predict(X_test)

In [14]:
from python.utils import write_submission

write_submission(y_pred, 'submissions')

Submission toy_submission.csv saved in submissions/toy_submission.csv.
